In [ ]:
import numpy as np
import pickle
import itertools
from statsmodels.stats.contingency_tables import mcnemar
from sklearn.metrics import balanced_accuracy_score

In [ ]:
def contingency_table(y_true, y_pred1, y_pred2):
    table = np.zeros((2, 2))
    for yt, yp1, yp2 in zip(y_true, y_pred1, y_pred2):
        if yp1 == yt and yp2 == yt:
            table[0, 0] += 1
        elif yp1 == yt and yp2 != yt:
            table[0, 1] += 1
        elif yp1 != yt and yp2 == yt:
            table[1, 0] += 1
        else:
            table[1, 1] += 1
    return table

In [ ]:
file_path = 'predicciones_modelos.pkl'
with open(file_path, 'rb') as file:
        models_predictions = pickle.load(file)
        #print(f"Datos cargados desde {file_path}")
y_test = models_predictions['y_test']

# Calcular balanced accuracy para cada modelo
balanced_accuracies = {}
for model_name, y_pred in models_predictions.items():
    if model_name != 'y_test':
        balanced_accuracies[model_name] = balanced_accuracy_score(y_test, y_pred)

print('Balanced acurracies \n' + str(balanced_accuracies) + '\n\n McNemar')

results_mcnemar = {}
# Comparación par a par de los modelos utilizando el test de McNemar
for model1_name, model2_name in itertools.combinations(models_predictions.keys(), 2):
    if model1_name == 'y_test' or model2_name == 'y_test':
        continue

    pred1 = models_predictions[model1_name]
    pred2 = models_predictions[model2_name]

    # Calcula la tabla de contingencia
    contingency_tab = contingency_table(y_test, pred1, pred2)

    # Realiza el test de McNemar
    mcnemar_result = mcnemar(contingency_tab, exact=False, correction=True)

    # Guarda los resultados
    results_mcnemar[(model1_name, model2_name)] = mcnemar_result

# Imprime los resultados
for (model1_name, model2_name), result in results_mcnemar.items():
    print(f"Comparación entre {model1_name} y {model2_name}:")
    print("Estadístico de McNemar:", result.statistic)
    print("Valor p:", result.pvalue)
    print()

Balanced acurracies 
{'DT': 0.975, 'SVM': 0.8477011494252874, 'XGB': 0.9833333333333334, 'RF': 0.975}

 McNemar
Comparación entre DT y SVM:
Estadístico de McNemar: 9.333333333333334
Valor p: 0.0022502265680857947

Comparación entre DT y XGB:
Estadístico de McNemar: 0.0
Valor p: 1.0

Comparación entre DT y RF:
Estadístico de McNemar: inf
Valor p: 0.0

Comparación entre SVM y XGB:
Estadístico de McNemar: 12.5
Valor p: 0.00040695201744495946

Comparación entre SVM y RF:
Estadístico de McNemar: 9.333333333333334
Valor p: 0.0022502265680857947

Comparación entre XGB y RF:
Estadístico de McNemar: 0.0
Valor p: 1.0



c:\Users\Mateo\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\stats\contingency_tables.py:1348: RuntimeWarning: divide by zero encountered in scalar divide
  statistic = (np.abs(n1 - n2) - corr)**2 / (1. * (n1 + n2))
